In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
import time
import random
import os

In [2]:
def generate_date_list(start, end):
    date_list = []
    curr = end
    while curr >= start:
        date_list.append(curr.strftime("%Y.%m.%d"))
        curr -= timedelta(days=1)
    return date_list

In [3]:
def extract_urls(json_data):
    urls = []
    for item in json_data.get('collection', []):
        html_str = item.get('html', '')
        soup = BeautifulSoup(html_str, 'html.parser')
        links = soup.find_all('a', href=True)
        for link in links:
            if 'n.news.naver.com' in link['href']:
                urls.append(link['href'])
    unique_urls = list(set(urls))
    return unique_urls

In [4]:
def get_urls_bydate(date, office_id):
    collected_urls = []
    base_url = "https://s.search.naver.com/p/newssearch/3/api/tab/more"
    headers = {'User-Agent': 'Mozilla/5.0', 'Referer': 'https://search.naver.com/'}
    for start in range(1, 2000, 10):
        params = {
        'abt': 'null',
        'de': date,
        'ds': date,
        'field': '0',
        'is_dts': '0',
        'is_sug_officeid': '0',
        'mynews': '1',
        'news_office_checked': office_id,
        'nqx_theme': '{"theme":{"sub":[{"name":"finance"}]}}',
        'nso': f'so:r,p:from{date.replace(".","")}to{date.replace(".","")},a:all',
        'office_category': '0',
        'office_section_code': '3',
        'office_type': '1',
        'pd': '3',
        'photo': '0',
        'query': '금리',
        'rev': '0',
        'service_area': '0',
        'sm': 'tab_smr',
        'sort': '2',
        'spq': '0',
        'ssc': 'tab.news.all',
        'start': start
        }
        try:
            res = requests.get(base_url, headers=headers, params=params, timeout=10)
            if res.status_code != 200 or not res.text.strip():
                print(f"[{date}] 수집 종료")
                break
            data = res.json()
            page_urls = extract_urls(data)
            
            if not page_urls:
                break
            
            collected_urls.extend(page_urls)
            
            time.sleep(random.uniform(0.3, 0.6))
            
        except Exception as e:
            print(f"[{date}] 에러 발생: {e}")
            raise e
            
    return list(set(collected_urls))

In [5]:
# 뉴스 본문 수집
def news_content(url):
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'Referer': 'https://news.naver.com/'
    }
    try:
        res = requests.get(url, headers=headers, timeout=15)
        if res.status_code != 200:
            return None
        soup = BeautifulSoup(res.text, 'html.parser')
        
        title = soup.select_one("h2#title_area")
        title = title.get_text(strip=True) if title else "제목 없음"
        
        date_tag = soup.select_one("span.media_end_head_info_datestamp_time")
        if date_tag and date_tag.has_attr('data-date-time'):
            date_str = date_tag['data-date-time']
        else:
            date_str = date_tag.get_text(strip=True) if date_tag else "날짜 없음"
        content = soup.select_one("#newsct_article") # 또는 "#dic_area"
        
        if content:
            content = content.get_text(" ", strip=True)
        else:
            content = "본문 없음"
        return {'title': title, 'date': date_str, 'content': content, 'url': url}

    except Exception as e:
        print(f"\n[{url}] 수집 중 에러 발생: {e}") 
        raise e

In [6]:
offices = {"매일경제": "1009", "한국경제": "1015", "머니투데이": "1008"}
keyword = "금리"
s_dt = datetime(2025, 12, 20) 
e_dt = datetime(2025, 12, 30)

In [7]:
# 한 달씩 끊어서 수집
from dateutil.relativedelta import relativedelta

current_month_end = e_dt
while current_month_end >= s_dt:
    current_month_start = current_month_end.replace(day=1)
    if current_month_start < s_dt:
        current_month_start = s_dt
    s_str = current_month_start.strftime("%Y%m%d")
    e_str = current_month_end.strftime("%Y%m%d")

    # URL 수집
    start_time = time.time()
    for office in offices:
        office_id = offices[office]
        file_name = f'news_urls_{office}.csv'

        seen_urls = set()
        if os.path.exists(file_name):
            existing_df = pd.read_csv(file_name)
            seen_urls = set(existing_df['url'].tolist())
            print(f"✅ {office}: 기존 파일에서 {len(seen_urls)}개의 주소를 확인했습니다.")
        
        target_dates = generate_date_list(current_month_start, current_month_end)
        for i, date in enumerate(target_dates):
            print(f"\n{office} [진행도: {i+1}/{len(target_dates)}] {date} 수집 중...")
            urls = get_urls_bydate(date, office_id)
            time.sleep(random.uniform(1.0, 2.0))
            
            if urls:
                new_urls = [u for u in urls if u not in seen_urls]
                if new_urls:
                    df = pd.DataFrame(new_urls, columns=['url'])
                    df['date'] = date
                    if not os.path.exists(file_name):
                        df.to_csv(file_name, index=False, encoding='utf-8-sig')
                    else:
                        df.to_csv(file_name, index=False, encoding='utf-8-sig', mode='a', header=False)
                        print(f"{office} {date} 저장 완료({len(new_urls)}개 추가됨)")
                        seen_urls.update(new_urls)
                else:
                    print(f"{office} {date}: 모두 중복되어 건너뜁니다.")
            else:
                print(f"{office} {date}: 검색 결과 없음.")
        print(f"\n✨ {office} {s_str} ~ {e_str} url 수집 완료")
        
        # content 수집
        input_file = f"news_urls_{office}.csv"
        output_file = f"news_contents_{office}.csv"
        max_workers = 4

        df_input = pd.read_csv(input_file)
        full_urls = set(df_input['url'].tolist())

        already_done_urls = set()
        if os.path.exists(output_file):
            df_existing = pd.read_csv(output_file)
            already_done_urls = set(df_existing['url'].tolist())
            print(f"기존 파일에서 {len(already_done_urls)}개의 수집 완료 데이터를 찾았습니다.")

        urls_to_crawl = [u for u in full_urls if u not in already_done_urls]
        print(f"수집해야 할 기사는 {len(urls_to_crawl)}개입니다.")

        results = []
        

        for i, url in enumerate(urls_to_crawl):
            try:
                data = news_content(url)
                if data:
                    results.append(data)
            except Exception as e:
                print(f"❌ 에러 발생: {e}")
            # 100개마다 중간 저장
            if (i + 1) % 100 == 0:
                pd.DataFrame(results).to_csv(output_file, index=False, encoding='utf-8-sig', 
                                            mode='a', header=not os.path.exists(output_file))
                results = [] # 저장 후 메모리 비우기
                print(f"✅{office} {i + 1}개 완료 및 중간 저장됨...")
                
                # 차단 방지
                # time.sleep(1)
                    
        # 남은 데이터 저장
        if results:
            pd.DataFrame(results).to_csv(output_file, index=False, encoding='utf-8-sig', 
                                        mode='a', header=not os.path.exists(output_file))

        print(f"\n🎉{office} {s_str} ~ {e_str} 내용 수집 완료! '{output_file}'을 확인하세요.")
    current_month_end = current_month_start - timedelta(days=1)

end_time = time.time()

# 3. 결과 계산
total_duration = end_time - start_time
print(f"\n⏱️ 싱글 스레드 총 소요 시간: {total_duration:.2f}초")
print(f"📄 기사 1개당 평균 소요 시간: {total_duration / len(urls_to_crawl):.2f}초")


매일경제 [진행도: 1/11] 2025.12.30 수집 중...

매일경제 [진행도: 2/11] 2025.12.29 수집 중...
매일경제 2025.12.29 저장 완료(23개 추가됨)

매일경제 [진행도: 3/11] 2025.12.28 수집 중...
매일경제 2025.12.28 저장 완료(12개 추가됨)

매일경제 [진행도: 4/11] 2025.12.27 수집 중...
매일경제 2025.12.27 저장 완료(4개 추가됨)

매일경제 [진행도: 5/11] 2025.12.26 수집 중...
매일경제 2025.12.26 저장 완료(10개 추가됨)

매일경제 [진행도: 6/11] 2025.12.25 수집 중...
매일경제 2025.12.25 저장 완료(15개 추가됨)

매일경제 [진행도: 7/11] 2025.12.24 수집 중...
매일경제 2025.12.24 저장 완료(11개 추가됨)

매일경제 [진행도: 8/11] 2025.12.23 수집 중...
매일경제 2025.12.23 저장 완료(11개 추가됨)

매일경제 [진행도: 9/11] 2025.12.22 수집 중...
매일경제 2025.12.22 저장 완료(11개 추가됨)

매일경제 [진행도: 10/11] 2025.12.21 수집 중...
매일경제 2025.12.21 저장 완료(10개 추가됨)

매일경제 [진행도: 11/11] 2025.12.20 수집 중...
매일경제 2025.12.20 저장 완료(6개 추가됨)

✨ 매일경제 20251220 ~ 20251230 url 수집 완료
수집해야 할 기사는 126개입니다.
✅매일경제 100개 완료 및 중간 저장됨...

🎉매일경제 20251220 ~ 20251230 내용 수집 완료! 'news_contents_매일경제.csv'을 확인하세요.

한국경제 [진행도: 1/11] 2025.12.30 수집 중...

한국경제 [진행도: 2/11] 2025.12.29 수집 중...
한국경제 2025.12.29 저장 완료(21개 추가됨)

한국경제 [진행도: 3/11] 2025.1

In [8]:
df1 = pd.read_csv('news_contents_매일경제.csv')
df2 = pd.read_csv('news_contents_한국경제.csv')
df3 = pd.read_csv('news_contents_머니투데이.csv')

print("전체 데이터 개수:", len(df1) + len(df2) + len(df3) - 3)

전체 데이터 개수: 405
